In [ ]:
# Import necessary libaries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from requests import get

In [ ]:
# Make a request
url = 'https://covid19.ncdc.gov.ng'
response = get(url, headers = {"Accept-Language": "en-US, en;q=0.5"})

In [ ]:
print(response.text[:500])

In [ ]:
# Create a beautifulsoup object
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

In [ ]:
# Locate table
covid_table = html_soup.find('div', class_ = 'table-responsive')

In [ ]:
# Scrape table headers and pass into a list
table_heading = covid_table.thead.find_all('th')
headings = []
for th in table_heading:
    headings.append(th.text)
print(headings)

In [ ]:
# Scrape table rows and pass into a list
table_rows = []
for i in covid_table.tbody.find_all('tr'):
    table_rows.append(i.text.replace('\n', ' ').strip())
table_rows

In [ ]:
# Format table rows for dataframe
table_rows = [i.split() for i in table_rows]
table_rows

In [ ]:
# Join 'Akwa' and 'Ibom'
del table_rows[27][1]
table_rows[27][0] = 'Akwa Ibom'
table_rows

In [ ]:
# Pass table rows into data frame
data = pd.DataFrame(table_rows)
data.head()

In [ ]:
# Set table headers from headings list
data.columns = headings
data.head()

In [ ]:
# Remove all commas from columns
data.replace(',','', regex=True, inplace=True)

In [ ]:
col = data.select_dtypes(object).columns[1:]
data[col] = data[col].apply(pd.to_numeric, errors='coerce')

In [ ]:
data.dtypes

In [ ]:
# Rename FCT to Abuja FCT
data.loc[data['States Affected'] == 'FCT', 'States Affected'] = 'Federal Capital Territory'

In [ ]:
data.columns = ['States', 'Cases', 'Infected', 'Recoveries', 'Deaths']

In [ ]:
data.head(3)

In [ ]:
# Export data frame to csv
data.to_csv('covid_nigeria.csv')

In [ ]:
# Import libararies for map plotting
import geopandas as gpd
import geoplot as gpt
import matplotlib.pyplot as plt

In [ ]:
# Load SHP files
shp = r'shapefile/Nigeria_Poly.shp'
shapefile = gpd.read_file(shp)

In [ ]:
type(shapefile), shapefile.shape

In [ ]:
shapefile.columns

In [ ]:
# Drop unecassary columns
shapefile = shapefile.drop(['id', 'country', 'name', 'enname', 'offname', 'boundary',
       'adminlevel', 'wikidata', 'wikimedia', 'timestamp', 'note', 'path',
       'rpath', 'iso3166_2'], axis=1)

In [ ]:
shapefile.columns = ['States', 'geometry']

In [ ]:
shapefile.head(2)

In [ ]:
# Merge GeoDataFrame with covid statistics data
data = shapefile.merge(data, left_on='States', right_on='States')

In [ ]:
type(data), data.shape

In [ ]:
data.head(2)

In [ ]:
data.head(3)

In [ ]:
# Define plot function
from jupyterthemes import jtplot
def visual(data, hue, cmap, theme):
    #context: 'paper' (Default), 'notebook', 'talk', or 'poster'
    #theme: 'oceans16', 'grade3', 'chesterish', 'onedork', 'monokai', 'solarizedl', 'solarizedd'
    jtplot.style(theme=theme, context='poster', grid=True, gridlines='-', ticks=True, spines=True, fscale=1.2) #figsize=(8.0, 7.0))
    fig, ax = plt.subplots(1, figsize=(38, 13))
    ax.set_title('Coronavirus {} by States'.format(hue), fontdict={'fontsize': '20', 'fontweight' : '3'})
    gpt.choropleth(data, hue=hue, cmap=cmap,
                   legend=True,edgecolor='lightgray', linewidth=0.1,
                   ax=ax,legend_kwargs={'aspect':30, 'fraction':0.036,'pad':0.00, 'orientation':'horizontal'});

    data['coords'] = data['geometry'].apply(lambda x: x.representative_point().coords[:])
    data['coords'] = [coords[0] for coords in data['coords']]

    for idx, row in data.iterrows():
        ax.annotate(s=row['States'], xy=row['coords'], color='black', fontsize=13, horizontalalignment='center')
        plt.savefig('{}.png'.format(hue))

In [ ]:
# Plot geomap for cases
visual(data, 'Cases', 'RdYlBu', theme)

In [ ]:
# Plot geomap for recoveries
visual(data, 'Recoveries', 'Blues', theme)

In [ ]:
# Plot geomap for deaths
visual(data, 'Deaths', 'YlGnBu', theme)